# Bachelor Thesis - Impacts in viscoelastic foams
# Modelisation of the problem

We will consider different models for impacts in viscoelastic foams. We will compare their reactions to impacts against rigid solids, and then against a similar viscoelastic solid.

## Part 1: Considering Different Models

We will consider at first simple models for impacts in visocelastic foams. We can consider 4 different models, and analyse their responses to an impact. The first two will be the Kelvin-Voigt, and Maxwell Model, while the latter two will be the Maxwell and Kelvin-Voigt representations of the standard linear model.

### Maxwell Model

The Maxwell model is described by a damper and sprring mounted in series, as seen in the diagram below
![title](images/maxwell.png)
We can see that following a deformation, the damper element will not return to it's initial length, and therre is therefore an element of permanent deformation. We can therefore discount this model, as it clearly does not represent the situation wee are trying to model, that of a boxing glove.

### Kelvin-Voigt Model
The Kelvin-Voigt model is described by a damper and spring mounted in parallel to one another, as seen in the diagram below.
![title](images/kelvin-voigt.png)
We can see that unlike the Maxwell model, when the Kelvin-Voigt system is compressed and released, it will return to its initial size. This is representative of the system we are seeking to model, and can therefore be explored further.

### Standard Linear Solid Model
The standard linear solid model (SLSM) can be represented in one of two ways, however both of these ways overcome the inability of the basic Maxwell model to return to its initial size when unloaded. We will consider both models below.
#### Maxwell representation of the SLSM
The maxwell representation of the standard linear solid model is described as a spring in parallel with a basic Maxwell model (i.e. a spring and damper in series) as described in the diagram below.
![title](images/standard-maxwell.png)

#### Kelvin-Voigt representation of the SLSM
The Kelvin-Voigt representation of the standard linear solid model is described as a spring in series with a basic Kelvin-Voigt model (i.e. a spring and damper in parallel), as described in the diagram below.
![title](images/standard-kelvin-voigt.png)

We will therefore consider the latter three models, ignoring only the basic maxwell model.

## Part 2: Equations behind the Models
$\renewcommand{\eps}{\varepsilon}$
We will now seek to write the equations for each of these three models. We will write out the differential equations in full, and we will write them iteratively for a given time step up to a first order approximation, in order to implement them programatically. We will write as $k_i$ the spring constants, and $\eta_i$ the dmaping constants, with $\sigma = \frac{F}{A}$ the stress, and $\eps = \frac{\Delta L}{L}$ the strain, where $F$ is the force, $A$ the surface area, and $L$ the length.
### Kelvin-Voigt

Differential Equation: $\sigma = k\eps + \eta \dot{\eps}$

Iterative equation: $\sigma_{t+\delta t} = \sigma{t} + \delta t (k \eps_t + \eta \dot{\eps}_t)$

### Maxwell representation of the standard linear solid model

Differential Equation: $\sigma + \frac{\eta}{k_2}\dot{\sigma} = k_1\eps + \frac{\eta(k_1 + k_2)}{k_2}\dot{\eps}$

Iterative equation: 

### Kelvin-Voigt representation of the standard linear solid model

Differential Equation: $\sigma + \frac{\eta}{k_1+k_2}\dot{\sigma} = \frac{k_1k_2}{K-1+k_2}\eps + \frac{k_1\eta}{k_1+k2}\dot{\eps}$

Iterative equation: 

#### General remarks on iterative modelling

For all three cases, when treating the model iteratively, we will have

$\eps_{t+\delta t} = \eps_t + \delta t \dot{\eps}_t$

$\dot{\eps}_{t+\delta t} = \dot{\eps}_{t} + \delta t \ddot{\eps}_t$

$\ddot{\eps}_{t+\delta t} = \frac{A}{m}\sigma_{t+\delta t}$


## Part 3: Implementing the models - Basic case:
We are interested in the behaviour of the model immediately after an impact. We will consider the basic case of a visco-eelastic solid dropped from a height, such that it hits a rigid surface with a velocity $v_0$ downwards. We therefore initialise $\dot{\eps}(t=0) =0$, and all other parameters are initialised to 0.


In [ ]:
### Import Necessary packages
import numpy as np
import matplotlib.pyplot as plt

### Kelvin-Voigt

In [ ]:
### Variable Initialisation


### References

https://en.wikipedia.org/wiki/Maxwell_material 

https://en.wikipedia.org/wiki/Kelvin-Voigt_material 

https://en.wikipedia.org/wiki/Standard_linear_solid_model 